In [1]:
from MBDOE import *
from generalize_functions import CEOSModels


[    0.00] Initializing mpi-sppy


## Choose experiments and model

In [2]:
model_option = "PR"
if_poly = False

if model_option=="PR":
    if not if_poly:
        from hfc32_emimtf2n_PR import configuration
    else:
        from hfc32_emimtf2n_PR_polynomial import configuration
        
elif model_option=="SRK":
    if not if_poly:
        from hfc32_emimtf2n_SRK import configuration
    else:
        from hfc32_emimtf2n_SRK_polynomial import configuration
        

In [3]:
# Experiments 
data_file = './emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv'

path = './emimtf2n/R32/Final_Results/MBDoE/Params/'

# Decide on model 
#para_file = path + 'PR_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'PR_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_1param_Opt2_GF.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'PR_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'PR_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'PR_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'PR_params_noTdep.csv'
#PR_type_opt = "No"

#para_file = path + 'PR_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

# important model
para_file = path + 'PR_params_quadTdep.csv'
PR_type_opt = "Quadratic"

#para_file = path + 'SRK_params_1param_Opt1.csv'
#PR_type_opt = "1Param_Opt1"

#para_file = path + 'SRK_params_1param_Opt2.csv'
#PR_type_opt = "1Param_Opt2"

#para_file = path + 'SRK_params_3params_Opt1.csv'
#PR_type_opt = "3Params_Opt1"

#para_file = path + 'SRK_params_3params_Opt2.csv'
#PR_type_opt = "3Params_Opt2"

#para_file = path + 'SRK_params_linTdep.csv'
#PR_type_opt = "Linear"

#para_file = path + 'SRK_params_noTdep.csv'
#PR_type_opt = "No"


#para_file = path + 'SRK_params_polyTdep.csv'
#PR_type_opt = "Polynomial"

#para_file = path + 'SRK_params_quadTdep.csv'
#PR_type_opt = "Quadratic"



model_creation = CEOSModels(para_file, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N",  Model_type = model_option, Tdep_type=PR_type_opt)

## Run pyomo.doe

In [ ]:
data_exp = pd.read_csv(data_file)
print(len(data_exp))
print(data_exp.iloc[1])

In [ ]:
print(model_creation.param_name_dict)

In [ ]:
mbdoe_obj = MBDOE(data_file, model_creation)

exp_set = range(len(data_exp))
#exp_set = range(12)
#exp_set = [0,1,2,3]

#exp_set = 0
# init_T: 283-348; 323 is good; init_pressure: low mole fraction use 150000; high 399800; x_option: 0.4-0.6;0.3-0.55;
# SRK: init_temp_option=323, init_pressure_option= 399800, init_x_c1_option = 0.5

total_fim = mbdoe_obj.sumDOE(exp_set, scale_opt=True, record_name="SRK_polyTdep", 
                             init_temp_option=343, 
                             init_pressure_option= 150000,
                             init_x_c1_option = 0.4,
                             poly_option=True)
#total_fim = mbdoe_obj.doe(exp_set, scale=True)

In [ ]:
print(total_fim)
print(np.linalg.det(total_fim))
print(np.trace(total_fim))
print(np.linalg.eigvals(total_fim))

## Heatmap

In [4]:
# read prior

f = open('emimtf2n_FIM_info/PR_quadTdep.json')

data = json.load(f)

prior_info = data[0]['Total']

In [5]:
#data_exp = pd.read_csv(data_file)
#print(data_exp)

In [10]:
# model list 
#T_range = [273, 360]
#x_range = [0.1, 0.9]

T_range = [273, 293, 313, 333, 353, 360]
x_range = [0.1, 0.3, 0.5, 0.7, 0.9]

data_exp = {'pressure': 549300}

fixed_model_set = []

for i in range(len(T_range)):
    for j in range(len(x_range)):
        data_exp['temperature'] = T_range[i]
        data_exp['x_R32'] = x_range[j]
        data_exp['x_emimTf2N'] = 1-x_range[j]
    
        print(data_exp)
        
        model = model_creation.create_model(data_exp)
        
        fixed_model_set.append(model)

{'pressure': 549300, 'temperature': 273, 'x_R32': 0.1, 'x_emimTf2N': 0.9}
2022-10-18 19:22:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.3, 'x_emimTf2N': 0.7}
2022-10-18 19:22:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.5, 'x_emimTf2N': 0.5}
2022-10-18 19:22:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.7, 'x_emimTf2N': 0.30000000000000004}
2022-10-18 19:22:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
{'pressure': 549300, 'temperature': 273, 'x_R32': 0.9, 'x_emimTf2N': 0.09999999999999998}
2022-10-18 19:22:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution 

In [11]:
design_range_values = [T_range, x_range]

In [12]:
mbdoe_obj = MBDOE(data_file, model_creation)

all_fim = mbdoe_obj.run_grid_search(design_range_values, fixed_models=fixed_model_set, 
                                    prior_FIM = prior_info, 
                                    scale_opt=True)
                                    #, record_name="SRK_polyTdep", 
                                    # init_temp_option=343, 
                                    # init_pressure_option= 150000,
                                    # init_x_c1_option = 0.4,
                                    # poly_option=True)



parameter set: {'fs.properties.PR_kappa_A["R32", "emimTf2N"]': 0.3212980565927437, 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': 0.8429816989080334, 'fs.properties.PR_kappa_B["R32", "emimTf2N"]': -0.5532108602621684, 'fs.properties.PR_kappa_B["emimTf2N", "R32"]': -0.6100855368792957, 'fs.properties.PR_kappa_C["R32", "emimTf2N"]': 0.1862684932461356, 'fs.properties.PR_kappa_C["emimTf2N", "R32"]': -0.3935451934926509}
All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']
30  design vectors will be searched.
=======This is the  1 th iteration=======
Design variable values of this iteration: {'fs.F101.inlet.temperature': {0: 1}, 'fs.F101.inlet.pressure': {0: 1}, 'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1}, "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}
Sensitivity information is scaled by its corresponding parameter nominal value.
After practice: {'fs.state_block.pressure': [0]}
name: fs.state_block.pressure
Build time with direct kaug mode [s]: 

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.9604644775390625e-06
Solve time with direct kaug mode [s]: 0.49323320388793945
Total wall clock time [s]: 0.49382805824279785
Existed information has been added.
FIM: [[ 4.58893995e+13 -1.12354961e+13  8.39404665e+13 -8.28060852e+12
   3.01881755e+13 -5.46625611e+12]
 [-1.12354961e+13  4.50335272e+12 -1.97045074e+13  3.19775595e+12
  -6.79102920e+12  2.03031387e+12]
 [ 8.39404665e+13 -1.97045074e+13  1.54372793e+14 -1.45936555e+13
   5.58144378e+13 -9.68287227e+12]
 [-8.28060852e+12  3.19775595e+12 -1.45936555e+13  2.27788550e+12
  -5.05529653e+12  1.45118175e+12]
 [ 3.01881755e+13 -6.79102920e+12  5.58144378e+13 -5.05529653e+12
   2.02855694e+13 -3.37188942e+12]
 [-5.46625611e+12  2.03031387e+12 -9.68287227e+12  1.45118175e+12
  -3.37188942e+12  9.27862908e+11]]
Trace: 228256863001733.3
Determinant: 4.82501178588089e+61
Condition number: 155230821.55467254
Minimal eigen value: 1448342.9646132595
Eigen values: [2.248

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.9141387939453125e-06
Solve time with direct kaug mode [s]: 0.4974040985107422
Total wall clock time [s]: 0.49796295166015625
Existed information has been added.
FIM: [[ 4.87033302e+13 -1.26210276e+13  8.87384051e+13 -9.28730958e+12
   3.17870529e+13 -6.11699243e+12]
 [-1.26210276e+13  4.95341374e+12 -2.20997861e+13  3.54767509e+12
  -7.59929233e+12  2.26974918e+12]
 [ 8.87384051e+13 -2.20997861e+13  1.62548947e+14 -1.63307770e+13
   5.85376573e+13 -1.08038812e+13]
 [-9.28730958e+12  3.54767509e+12 -1.63307770e+13  2.54651697e+12
  -5.64049228e+12  1.63314327e+12]
 [ 3.17870529e+13 -7.59929233e+12  5.85376573e+13 -5.64049228e+12
   2.11921523e+13 -3.74896433e+12]
 [-6.11699243e+12  2.26974918e+12 -1.08038812e+13  1.63314327e+12
  -3.74896433e+12  1.05010398e+12]]
Trace: 240994463865788.94
Determinant: 1.8486603032311946e+61
Condition number: 252841630.94897938
Minimal eigen value: 939649.4302471492
Eigen values: [2.37

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.198883056640625e-06
Solve time with direct kaug mode [s]: 0.5105435848236084
Total wall clock time [s]: 0.5111808776855469
Existed information has been added.
FIM: [[ 5.04832701e+13 -1.21998618e+13  9.23172698e+13 -9.05588924e+12
   3.31717373e+13 -6.01641456e+12]
 [-1.21998618e+13  4.29026756e+12 -2.15492230e+13  3.09553475e+12
  -7.47441587e+12  1.99633200e+12]
 [ 9.23172698e+13 -2.15492230e+13  1.69629785e+14 -1.60623398e+13
   6.12430444e+13 -1.07170868e+13]
 [-9.05588924e+12  3.09553475e+12 -1.60623398e+13  2.23975995e+12
  -5.59520308e+12  1.44871355e+12]
 [ 3.31717373e+13 -7.47441587e+12  6.12430444e+13 -5.59520308e+12
   2.22154540e+13 -3.74972554e+12]
 [-6.01641456e+12  1.99633200e+12 -1.07170868e+13  1.44871355e+12
  -3.74972554e+12  9.39981021e+11]]
Trace: 249798518003574.9
Determinant: 1.6451454378593666e+61
Condition number: 300082822.2526492
Minimal eigen value: 823201.4579694999
Eigen values: [2.470286

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 5.4836273193359375e-06
Solve time with direct kaug mode [s]: 0.6178929805755615
Total wall clock time [s]: 0.618455171585083
Existed information has been added.
FIM: [[ 4.86357916e+13 -1.11996685e+13  8.93311820e+13 -8.31558504e+12
   3.22491326e+13 -5.52920781e+12]
 [-1.11996685e+13  3.94200753e+12 -1.97878004e+13  2.83353731e+12
  -6.86928419e+12  1.82078049e+12]
 [ 8.93311820e+13 -1.97878004e+13  1.64911877e+14 -1.47617723e+13
   5.98310086e+13 -9.86350433e+12]
 [-8.31558504e+12  2.83353731e+12 -1.47617723e+13  2.04280211e+12
  -5.14962030e+12  1.31684654e+12]
 [ 3.22491326e+13 -6.86928419e+12  5.98310086e+13 -5.14962030e+12
   2.18124833e+13 -3.45818945e+12]
 [-5.52920781e+12  1.82078049e+12 -9.86350433e+12  1.31684654e+12
  -3.45818945e+12  8.51769742e+11]]
Trace: 242196730809759.9
Determinant: 1.3499740172019652e+61
Condition number: 283280486.7355584
Minimal eigen value: 845091.8242527295
Eigen values: [2.393980

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.152557373046875e-06
Solve time with direct kaug mode [s]: 0.504615306854248
Total wall clock time [s]: 0.5053141117095947
Existed information has been added.
FIM: [[ 4.60305031e+13 -1.08125786e+13  8.43732640e+13 -8.00354020e+12
   3.04054768e+13 -5.30503318e+12]
 [-1.08125786e+13  3.88940369e+12 -1.90452365e+13  2.79105379e+12
  -6.59076686e+12  1.79020047e+12]
 [ 8.43732640e+13 -1.90452365e+13  1.55484003e+14 -1.41632633e+13
   5.63280311e+13 -9.43360433e+12]
 [-8.00354020e+12  2.79105379e+12 -1.41632633e+13  2.00849324e+12
  -4.92517203e+12  1.29215174e+12]
 [ 3.04054768e+13 -6.59076686e+12  5.63280311e+13 -4.92517203e+12
   2.05120943e+13 -3.29700039e+12]
 [-5.30503318e+12  1.79020047e+12 -9.43360433e+12  1.29215174e+12
  -3.29700039e+12  8.33995742e+11]]
Trace: 228758493215042.0
Determinant: 1.4478223678110146e+61
Condition number: 240797728.73026347
Minimal eigen value: 938449.1769126306
Eigen values: [2.259764

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 7.62939453125e-06
Solve time with direct kaug mode [s]: 0.5794212818145752
Total wall clock time [s]: 0.5800783634185791
Existed information has been added.
FIM: [[ 1.00297544e+16 -1.08605868e+16  2.04374697e+16 -9.30475962e+15
   8.14439652e+15 -7.10555231e+15]
 [-1.08605868e+16  1.17943380e+16 -2.21376959e+16  1.01055750e+16
  -8.82443995e+15  7.71762824e+15]
 [ 2.04374697e+16 -2.21376959e+16  4.16478709e+16 -1.89667882e+16
   1.65977352e+16 -1.44841947e+16]
 [-9.30475962e+15  1.01055750e+16 -1.89667882e+16  8.65869725e+15
  -7.56060253e+15  6.61269834e+15]
 [ 8.14439652e+15 -8.82443995e+15  1.65977352e+16 -7.56060253e+15
   6.61493033e+15 -5.77382353e+15]
 [-7.10555231e+15  7.71762824e+15 -1.44841947e+16  6.61269834e+15
  -5.77382353e+15  5.05018866e+15]]
Trace: 8.379577947490462e+16
Determinant: 1.2130715995632623e+67
Condition number: 41146923405.39716
Minimal eigen value: 2035425.709836967
Eigen values: [8.375150

name: fs.state_block.pressure
Build time with direct kaug mode [s]: 6.67572021484375e-06
Solve time with direct kaug mode [s]: 0.6661398410797119
Total wall clock time [s]: 0.6667284965515137
Existed information has been added.
FIM: [[ 2.51878252e+15 -1.58226916e+15  5.22534171e+15 -1.38122747e+15
   2.12054553e+15 -1.07488100e+15]
 [-1.58226916e+15  1.00240093e+15 -3.28616686e+15  8.75344361e+14
  -1.33488511e+15  6.81403950e+14]
 [ 5.22534171e+15 -3.28616686e+15  1.08437999e+16 -2.86915120e+15
   4.40182751e+15 -2.23312517e+15]
 [-1.38122747e+15  8.75344361e+14 -2.86915120e+15  7.64492707e+14
  -1.16566074e+15  5.95175358e+14]
 [ 2.12054553e+15 -1.33488511e+15  4.40182751e+15 -1.16566074e+15
   1.78724173e+15 -9.07370137e+14]
 [-1.07488100e+15  6.81403950e+14 -2.23312517e+15  5.95175358e+14
  -9.07370137e+14  4.63397359e+14]]
Trace: 1.738011516359682e+16
Determinant: 1.407087198899804e+66
Condition number: 7590619167.791995
Minimal eigen value: 2287721.051745439
Eigen values: [1.7365

In [13]:
fixed = {}

all_fim.extract_criteria()
print(all_fim.store_all_results_dataframe)

all_fim.figure_drawing(fixed, ['Temperature','x_R32'], ' ','Temperature [K]', 'x (R32)' )


Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.7)
Design variable:  ['Temperature', 'x_R32']
Value          :  (273, 0.9)
Design variable:  ['Temperature', 'x_R32']
Value          :  (293, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (293, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (293, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (293, 0.7)
Design variable:  ['Temperature', 'x_R32']
Value          :  (293, 0.9)
Design variable:  ['Temperature', 'x_R32']
Value          :  (313, 0.1)
Design variable:  ['Temperature', 'x_R32']
Value          :  (313, 0.3)
Design variable:  ['Temperature', 'x_R32']
Value          :  (313, 0.5)
Design variable:  ['Temperature', 'x_R32']
Value          :  (31